### Load the model

In [1]:
import tensorflow as tf
import numpy
import h5py
from qkeras.utils import _add_supported_quantized_objects
from qkeras import quantized_bits
co = {}; _add_supported_quantized_objects(co)

model_path = f"/eos/project/c/cms-l1ml/public/Pawel/axol1tl_model.h5"

model = tf.keras.models.load_model(model_path, custom_objects=co)
#model_config = model.get_config()

2025-08-20 12:24:12.684014: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-20 12:24:12.721100: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-20 12:24:12.721157: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-20 12:24:12.721202: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-20 12:24:12.732826: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-20 12:24:12.733322: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
data_path = f"/eos/project/c/cms-l1ml/public/Pawel/complete.h5"
data_file = h5py.File(data_path, 'r')

### Copy weights into the model

In [3]:
#model.compile()

ds_weights = data_file['model']['trimmed_encoder']['model_weights']
for layer in iter(model.layers):
    #print(layer.name)
    if layer.name in ('input_1','subtract','dot'): continue

    if layer.name == 'q_dense_4':
        weights_group = ds_weights['q_dense_4']['variational_auto_encoder']['vae__encoder']['q_dense_4']
        layer.set_weights((weights_group['kernel:0'], weights_group['bias:0']))
        continue

    weights_group = ds_weights[layer.name][layer.name]
    layer.set_weights((weights_group['kernel:0'], weights_group['bias:0']))

### Prepare data for tests

In [4]:
input_quantizer = quantized_bits(14, 5, alpha=1)
output_quantizer = quantized_bits(18,13,alpha=1)
bias_quantizer = quantized_bits(18,12,alpha=1)

In [5]:
X = data_file['data']["Background_data"]["Test"]["DATA"][:2048]

X_flat = X.reshape(X.shape[0], -1)
X_flat = input_quantizer(X_flat)
X_flat.dtype

tf.float32

### Fragmentary model

In [6]:
inputs = tf.keras.layers.Input(shape=model.layers[0].input_shape[0])

layer_list = [inputs]

# QDense layers
for i in range(1,9):
    layer_list.append(model.layers[i](layer_list[i-1]))

# Subtract and Dot
layer_list.append(model.layers[9]((layer_list[8],layer_list[2])))
layer_list.append(model.layers[10]((layer_list[9],layer_list[9])))

fragm_model = tf.keras.Model(inputs=inputs, outputs=layer_list[-1], name='fragm_model')
fragm_model.summary()

Model: "fragm_model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None, 57)]           0         []                            
                                                                                                  
 q_dense (QDense)            multiple                     1682      ['input_1[0][0]']             
                                                                                                  
 q_dense_1 (QDense)          multiple                     300       ['q_dense[1][0]']             
                                                                                                  
 q_dense_2 (QDense)          multiple                     99        ['q_dense_1[1][0]']           
                                                                                        

In [ ]:
output_quantizer(fragm_model.predict(X_flat))

In [ ]:
fragm_model.layers

### Background prediction (test)

In [ ]:
X = data_file['data']["Background_data"]["Test"]["DATA"][:2048]
scale_data = data_file['data']['Normalisation']['norm_scale'][:].flatten()
offset_data =  data_file['data']['Normalisation']['norm_bias'][:].flatten()

X_flat = X.reshape(X.shape[0], -1)
X_int = (X_flat * scale_data) + offset_data
X_flat_int = X_int.astype('int')

In [ ]:
input_quantizer = quantized_bits(14, 5, alpha=1)
output_quantizer = quantized_bits(18,13,alpha=1)
bias_quantizer = quantized_bits(18,12,alpha=1)

In [ ]:
from tqdm import tqdm
batch_size = 128
predictions = []

for start in tqdm(range(0, 1024, batch_size), desc="Qkeras preds"):
        end = min(start + batch_size, X_flat_int.shape[0])
        X_for_qk = X_flat_int[start:end].astype('float')
        X_biased = bias_quantizer(X_for_qk - offset_data).numpy().astype('float')
        X_scaled = bias_quantizer(X_biased / scale_data)
        X_scaled_quantized_2 = input_quantizer(X_scaled)
        
        pred = model.predict(X_scaled_quantized_2)
        quantized_pred = output_quantizer(pred)
        predictions.append(quantized_pred)

In [ ]:
predictions[0]

### Conversion into a hls4ml model

In [ ]:
import hls4ml

hls_config = hls4ml.utils.config_from_keras_model(
    model,
    granularity='name', 
    default_precision='fixed<14,7>',
    backend='Vitis')
print("-----------------------------------")


In [ ]:
hls_model = hls4ml.converters.convert_from_keras_model(
    model, hls_config=hls_config, backend='Vitis',
    output_dir='./hls_test'
)

In [ ]:
hls_model.compile()

### Predictions for HLS model

In [ ]:
from tqdm import tqdm
batch_size = 128
predictions_hls = []

for start in tqdm(range(0, 1024, batch_size), desc="hls preds"):
        end = min(start + batch_size, X_flat_int.shape[0])
        X_for_qk = X_flat_int[start:end].astype('float')
        X_biased = quantizer(X_for_qk - offset_data).numpy().astype('float')
        X_scaled = quantizer(X_biased / scale_data)
        X_scaled_quantized_2 = quantizer(X_scaled)
        
        pred = hls_model.predict(numpy.ascontiguousarray(X_scaled_quantized_2))
        quantized_pred = quantizer(pred)
        predictions_hls.append(quantized_pred)

In [ ]:
predictions[0][:,0].numpy()

In [ ]:
print(numpy.column_stack((predictions[0][:,0].numpy(),predictions_hls[0][:,0].numpy())))